In [3]:
from keras.preprocessing import  sequence
from keras.datasets import  imdb
from keras.models import  Sequential
from keras.layers import  Embedding
from keras.layers import LSTM, Dense, GRU
from keras.utils import print_summary
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
max_features = 20000
maxlen = 80
batch_size = 32

In [3]:
(x_train,y_train),(x_test,y_test) = imdb.load_data(num_words=max_features)
np.unique(y_test)

In [5]:
def calcute(data):
    re ={}
    for i in data:
        re.setdefault(i,0)
        re[i] +=1
    return re
calcute(y_test[:1000])

{0: 526, 1: 474}

In [6]:
# pad sequence
x_train = sequence.pad_sequences(x_train,maxlen=maxlen, padding="post")
x_test = sequence.pad_sequences(x_test,maxlen=maxlen, padding="pre") # 默认从前面截取和 填补

In [9]:
model = Sequential()
model.add(Embedding(max_features,128,input_length=maxlen))
model.add(GRU(128,dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(1,activation='sigmoid'))

In [12]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['acc'])
print('train..')

for i in range(2):
    history = model.fit(x_train, y_train, batch_size=batch_size, epochs=10, validation_data=(x_test, y_test))
    plt.plot(history.history["acc"])
    plt.plot(history.history["val_acc"])
    plt.title("Accuracy")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend(["train", "test"],loc="upper left")
    plt.show()
    score, acc = model.evaluate(x_test, y_test, batch_size=batch_size)
    print(score, acc)

train..
Train on 25000 samples, validate on 25000 samples
Epoch 1/10
 3872/25000 [===>..........................] - ETA: 1:11 - loss: 0.6634 - acc: 0.6090

KeyboardInterrupt: 

# convD1 一维卷积神经网络  对embeding不同处理的结果比较

In [106]:
from keras.layers import Dropout, Conv1D, GlobalMaxPooling1D, Activation, AveragePooling1D, MaxPooling1D
from keras.layers import Input, Flatten
from keras.models import  Model
import keras

In [134]:
max_features = 5000
maxlen = 200
batch_size = 128
embedding_dims = 50
filters = 250
kernel_size =3
hidden_dims = 250
epochs = 10

print("load data ...")
(train_x,train_y),(test_x,test_y) = imdb.load_data(num_words=max_features)

train_x = sequence.pad_sequences(train_x, maxlen=maxlen, padding="post")
test_x = sequence.pad_sequences(test_x, maxlen=maxlen, padding="post")
print(train_x.shape,test_x.shape)

load data ...
(25000, 200) (25000, 200)


In [133]:
import numpy as np
np.median([len(i) for i in train_x])

178.0

In [1]:
import keras.backend as K
from keras.layers import Conv2D, AveragePooling2D, MaxPooling2D,Subtract
import keras

In [137]:
inputs_1 = Input(shape=(200,))
x_embed = Embedding(input_dim=max_features, input_length=maxlen, 
                    output_dim=embedding_dims)(inputs_1)

def expand_x(x):
    x_embed = K.expand_dims(x, -1)
    return x_embed

x_embed = keras.layers.core.Lambda(expand_x)(x_embed)
#x_embed = keras.layers.core.RepeatVector(1)(x_embed) 不可用， 这里的pading 注意。

In [138]:
con_2 = Conv2D(128, kernel_size=(10, embedding_dims),data_format="channels_last",
               strides=(1, 1), padding="valid", activation="relu")(x_embed)
con_2_pool = MaxPooling2D(pool_size=(2, 1))(con_2)

con_2_pool_flat = Flatten()(con_2_pool)
output_2 = Dense(1, activation="sigmoid")(con_2_pool_flat)
print(output_2.shape)

model_1 = Model(inputs_1, output_2)
model_1.summary()

from keras.callbacks import ModelCheckpoint
savebestmodel = 'tf_4dim.h5'
checkpoint = ModelCheckpoint(savebestmodel, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks=[checkpoint] 

model_1.compile(loss="binary_crossentropy", optimizer="Adadelta", metrics=["acc"])
model_1.fit(train_x, train_y, batch_size=batch_size, epochs=epochs, validation_data=(test_x, test_y),
            verbose=1, callbacks=callbacks)

(?, 1)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_44 (InputLayer)        (None, 200)               0         
_________________________________________________________________
embedding_47 (Embedding)     (None, 200, 50)           250000    
_________________________________________________________________
lambda_6 (Lambda)            (None, 200, 50, 1)        0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 191, 1, 128)       64128     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 95, 1, 128)        0         
_________________________________________________________________
flatten_39 (Flatten)         (None, 12160)             0         
_________________________________________________________________
dense_47 (Dense)             (None, 1)                 12161     
Tot

In [119]:
model_1.history.history

{'acc': [0.99492,
  0.99848,
  0.99732,
  0.998,
  0.998720000038147,
  0.99956,
  0.99964,
  0.9996,
  0.9976,
  0.9998],
 'loss': [0.021607562263011932,
  0.012184968919456005,
  0.012973040844500065,
  0.011352410731464625,
  0.0074836928868293765,
  0.0054534724199771884,
  0.004550972958952189,
  0.004116294031441212,
  0.00977890340629965,
  0.002959204211831093],
 'val_acc': [0.87468,
  0.87424,
  0.87224,
  0.873,
  0.86952,
  0.872639999961853,
  0.872719999961853,
  0.87068,
  0.871759999961853,
  0.86652],
 'val_loss': [0.6045735818386078,
  0.6432584668159484,
  0.6620480927181244,
  0.6754186646938324,
  0.7101701876068115,
  0.7297660604286194,
  0.7535416314125061,
  0.7661286487960816,
  0.753690506286621,
  0.8445094299507141]}

In [139]:
inputs_2 = Input(shape=(200,))
x_embed_1 = Embedding(input_dim=max_features, input_length=maxlen, output_dim=embedding_dims)(inputs_2)

x_convd_1 = Conv1D(128, kernel_size=10, activation="relu")(x_embed_1)
x_convd_1_pool = MaxPooling1D()(x_convd_1)

con_1_pool_flat = Flatten()(x_convd_1_pool)
output_1 = Dense(1, activation="sigmoid")(con_1_pool_flat)
print(output_1.shape)

model_com = Model(inputs_2, output_1)
model_com.summary()
           
savebestmodel = 'keras_convd1.h5'
checkpoint = ModelCheckpoint(savebestmodel, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_com=[checkpoint] 

model_com.compile(loss="binary_crossentropy", optimizer="Adadelta", metrics=["acc"])
model_com.fit(train_x, train_y, batch_size=batch_size, epochs=epochs, validation_data=(test_x, test_y),
            verbose=1, callbacks=callbacks_com)

(?, 1)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_45 (InputLayer)        (None, 200)               0         
_________________________________________________________________
embedding_48 (Embedding)     (None, 200, 50)           250000    
_________________________________________________________________
conv1d_61 (Conv1D)           (None, 191, 128)          64128     
_________________________________________________________________
max_pooling1d_9 (MaxPooling1 (None, 95, 128)           0         
_________________________________________________________________
flatten_40 (Flatten)         (None, 12160)             0         
_________________________________________________________________
dense_48 (Dense)             (None, 1)                 12161     
Total params: 326,289
Trainable params: 326,289
Non-trainable params: 0
_______________________________________________________________

# cnn 多个 卷积组合的搞法。

In [34]:
# conv1D 类似与 4维中 kerner_size = (N, embedding_dims)(())
# (2,)*1 + (1, )
# (1, 2) + (2, 3)

In [44]:
inputs = Input(shape=(400,))
x_embed = Embedding(input_dim=max_features, output_dim=embedding_dims,input_length=maxlen)(inputs)
x_conved_1 = Conv1D(128, kernel_size=10, strides=5, padding="valid", activation="relu")(x_embed)
x_conved_1_drop = Dropout(0.2)(x_conved_1)
x_1_pool = AveragePooling1D(pool_size=2)(x_conved_1_drop)

x_conved_2 = Conv1D(128, kernel_size=15, strides=5, padding="valid", activation="relu")(x_embed)
x_conved_2_drop = Dropout(0.2)(x_conved_2)
x_2_pool = AveragePooling1D(pool_size=2)(x_conved_2_drop)
print(x_conved_2.shape)

x_conved_3 = Conv1D(128, kernel_size=20, strides=5, padding="valid", activation="relu")(x_embed)
x_conved_3_drop = Dropout(0.2)(x_conved_3)
x_3_pool = AveragePooling1D(pool_size=2)(x_conved_3_drop)
# pool是 向下取整用的

megerd = keras.layers.concatenate([x_1_pool, x_2_pool, x_3_pool], axis=1)
megerd = Flatten()(megerd)

output = Dense(1, activation="sigmoid")(megerd)

model = Model(inputs, output)
# model.summary()

(?, 78, 128)


In [45]:
model.compile(optimizer="Adadelta", loss="binary_crossentropy", metrics=["acc"])

model.fit(train_x, train_y, batch_size=batch_size, epochs=epochs,
          validation_data=(test_x,test_y), verbose=1)
# model.fit(x_train,y_train,batch_size=batch_size,validation_data=(x_test,y_test),epochs=2)
# score, acc = model.evaluate(test_x, test_y, batch_size=batch_size)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 62s 2ms/step - loss: 0.5379 - acc: 0.6988 - val_loss: 0.3582 - val_acc: 0.8400
Epoch 2/2
25000/25000 [==============================] - 12s 463us/step


In [41]:
embedding_dims = 50
model = Sequential()
model.add(Embedding(input_dim = max_features,output_dim=embedding_dims,input_length=maxlen))
model.add(Dropout(0.2))
model.add(Conv1D(filters=filters,kernel_size=kernel_size,padding='valid',activation='relu',strides=1))
model.add(GlobalMaxPooling1D())

model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary()
# model.compile(optimizer="Adadelta", loss="binary_crossentropy", metrics=["acc"])
# model.fit(x_train,y_train,batch_size=batch_size,validation_data=(x_test,y_test),epochs=2)
# score,acc = model.evaluate(test_x,test_y,batch_size=batch_size)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_18 (Embedding)     (None, 400, 50)           250000    
_________________________________________________________________
dropout_7 (Dropout)          (None, 400, 50)           0         
_________________________________________________________________
conv1d_44 (Conv1D)           (None, 398, 250)          37750     
_________________________________________________________________
global_max_pooling1d_4 (Glob (None, 250)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 250)               62750     
_________________________________________________________________
dropout_8 (Dropout)          (None, 250)               0         
_________________________________________________________________
activation_7 (Activation)    (None, 250)               0         
__________

In [15]:
print(score,acc)

0.2741204485177994 0.88688


# AUTO_ENCODER

In [ ]:
import numpy as np
np.random.seed(1337)  # for reproducibility
 
from keras.datasets import mnist
from keras.models import Model #泛型模型
from keras.layers import Dense, Input
import matplotlib.pyplot as plt
 
# X shape (60,000 28x28), y shape (10,000, )
(x_train, _), (x_test, y_test) = mnist.load_data()
 
# 数据预处理
x_train = x_train.astype('float32') / 255. - 0.5       # minmax_normalized
x_test = x_test.astype('float32') / 255. - 0.5         # minmax_normalized
x_train = x_train.reshape((x_train.shape[0], -1))
x_test = x_test.reshape((x_test.shape[0], -1))
print(x_train.shape)
print(x_test.shape)
 

   57344/11490434 [..............................] - ETA: 16:18:09

In [ ]:
# 压缩特征维度至2维
encoding_dim = 2
 
# this is our input placeholder
input_img = Input(shape=(784,))
 
# 编码层
encoded = Dense(128, activation='relu')(input_img)
encoded = Dense(64, activation='relu')(encoded)
encoded = Dense(10, activation='relu')(encoded)
encoder_output = Dense(encoding_dim)(encoded)
 
# 解码层
decoded = Dense(10, activation='relu')(encoder_output)
decoded = Dense(64, activation='relu')(decoded)
decoded = Dense(128, activation='relu')(decoded)
decoded = Dense(784, activation='tanh')(decoded)
 
# 构建自编码模型
autoencoder = Model(inputs=input_img, outputs=decoded)
 
# 构建编码模型
encoder = Model(inputs=input_img, outputs=encoder_output)
 
# compile autoencoder
autoencoder.compile(optimizer='adam', loss='mse')
 
# training
autoencoder.fit(x_train, x_train, epochs=20, batch_size=256, shuffle=True)
 
# plotting
encoded_imgs = encoder.predict(x_test)
plt.scatter(encoded_imgs[:, 0], encoded_imgs[:, 1], c=y_test, s=3)
plt.colorbar()
plt.show()